## Calculating Symbolic 3D expressions
$$ U = cos(\theta s /2) I + i sin(\theta s /2) R $$
Where $$ R_{xyz} = sin(\psi) cos(\phi) X + sin(\psi) sin(\phi) Y + cos(\psi) Z $$

In [1]:
import sympy as sp
import numpy as np
from numpy import exp, sqrt, array
from numpy.random import multivariate_normal

In [2]:
# === Symbols ===
theta, s, phi, psi = sp.symbols('theta s phi psi', real=True)
alpha = s * theta / 2

# === Pauli Matrices and Identity ===
I2 = sp.eye(2)
X = sp.Matrix([[0, 1], [1, 0]])
Y = sp.Matrix([[0, -sp.I], [sp.I, 0]])
Z = sp.Matrix([[1, 0], [0, -1]])

R = (
    sp.sin(psi) * sp.cos(phi) * X +
    sp.sin(psi) * sp.sin(phi) * Y +
    sp.cos(psi) * Z
)

U = sp.cos(alpha) * I2 - sp.I * sp.sin(alpha) * R
U_dag = U.H

---
## Symbolic Calculations ##

Find components in X Y Z

In [3]:
def pauli_decomposition(Z_s ,paulis):
    components = {}
    for label, sigma in paulis.items():
        coeff = (Z_s * sigma).trace() / 2
        components[label] = sp.simplify(coeff)
    return components

paulis = {'I': I2, 'X': X, 'Y': Y, 'Z': Z}


In [4]:
def fully_simplify_pauli_decomposition(decomp, collect_vars=None):
    """
    Deeply simplify and factor a symbolic Pauli decomposition dictionary.

    Parameters:
        decomp (dict): output from pauli_decomposition
        collect_vars (list): list of sympy symbols to factor/collect

    Returns:
        dict: simplified and factored expressions
    """
    simplified = {}
    for pauli, expr in decomp.items():
        # Step-by-step simplification
        expr = sp.expand_trig(expr)
        expr = sp.trigsimp(expr)
        expr = sp.expand(expr)
        expr = sp.powsimp(expr, force=True)
        expr = sp.simplify(expr)
        expr = sp.expand_complex(expr)

        # Collect φ, ψ, or any variables provided
        if collect_vars:
            for var in collect_vars:
                expr = sp.collect(expr, var)
                if isinstance(var, sp.Basic):
                    expr = sp.collect(expr, sp.exp(sp.I * var))
                    expr = sp.collect(expr, sp.exp(-sp.I * var))
        
        simplified[pauli] = expr
    return simplified


## Z(t) Component ##

In [5]:
Z_s = U_dag * Z * U
Z_s = sp.simplify(Z_s)
Z_s = sp.expand_trig(Z_s)
Z_s = sp.trigsimp(Z_s)
Z_s

Matrix([
[                                                -2*sin(psi)**2*sin(s*theta/2)**2 + 1, 2*I*(I*sin(phi) - cos(phi))*(I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))*sin(psi)*sin(s*theta/2)],
[-2*I*(I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))*exp(I*phi)*sin(psi)*sin(s*theta/2),                                                              2*sin(psi)**2*sin(s*theta/2)**2 - 1]])

In [6]:
Z_s_decomp = pauli_decomposition(Z_s, paulis)
Z_s_decomp = sp.simplify(Z_s_decomp)
Z_s_decomp = sp.trigsimp(Z_s_decomp)
Z_s_decomp

{I: 0, X: I*((I*sin(phi) - cos(phi))*(I*sin(s*theta/2)*cos(psi) + cos(s*theta/2)) - (I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))*exp(I*phi))*sin(psi)*sin(s*theta/2), Y: (-(I*sin(phi) - cos(phi))*(I*sin(s*theta/2)*cos(psi) + cos(s*theta/2)) + (-I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))*exp(I*phi))*sin(psi)*sin(s*theta/2), Z: -2*sin(psi)**2*sin(s*theta/2)**2 + 1}

In [7]:
Z_s_decomp = pauli_decomposition(Z_s, paulis)

# Clean and factor using φ and ψ
Z_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    Z_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
Z_s_decomp_cleaned

{'I': 0,
 'X': (-2*sin(phi)*cos(s*theta/2) + 2*sin(s*theta/2)*cos(phi)*cos(psi))*sin(psi)*sin(s*theta/2),
 'Y': (2*sin(phi)*sin(s*theta/2)*cos(psi) + 2*cos(phi)*cos(s*theta/2))*sin(psi)*sin(s*theta/2),
 'Z': -2*sin(psi)**2*sin(s*theta/2)**2 + 1}

Finding X component

In [8]:
Z_s_decomp_cleaned['X']


(-2*sin(phi)*cos(s*theta/2) + 2*sin(s*theta/2)*cos(phi)*cos(psi))*sin(psi)*sin(s*theta/2)

Finding Y component

In [9]:
Z_s_decomp_cleaned['Y']

(2*sin(phi)*sin(s*theta/2)*cos(psi) + 2*cos(phi)*cos(s*theta/2))*sin(psi)*sin(s*theta/2)

Finding Y component

In [10]:
Z_s_decomp_cleaned['Z']

-2*sin(psi)**2*sin(s*theta/2)**2 + 1

## Y(t) Component ##

In [11]:
Y_s = U_dag * Y * U
Y_s = sp.simplify(Y_s)
Y_s = sp.expand_trig(Y_s)
Y_s = sp.trigsimp(Y_s)
Y_s

Matrix([
[          2*(sin(phi)*sin(s*theta/2)*cos(psi) - cos(phi)*cos(s*theta/2))*sin(psi)*sin(s*theta/2), I*(-(I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))**2 + exp(-2*I*phi)*sin(psi)**2*sin(s*theta/2)**2)],
[I*((I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))**2 - exp(2*I*phi)*sin(psi)**2*sin(s*theta/2)**2),            2*(-sin(phi)*sin(s*theta/2)*cos(psi) + cos(phi)*cos(s*theta/2))*sin(psi)*sin(s*theta/2)]])

In [12]:
Y_s_decomp = pauli_decomposition(Y_s, paulis)
Y_s_decomp = sp.simplify(Y_s_decomp)
Y_s_decomp = sp.trigsimp(Y_s_decomp)
Y_s_decomp

{I: 0, X: -I*exp(2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2 + sin(s*theta)*cos(psi) + I*exp(-2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2, Y: -exp(2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2 - sin(s*theta/2)**2*cos(psi)**2 + cos(s*theta/2)**2 - exp(-2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2, Z: 2*(sin(phi)*sin(s*theta/2)*cos(psi) - cos(phi)*cos(s*theta/2))*sin(psi)*sin(s*theta/2)}

In [13]:
Y_s_decomp = pauli_decomposition(Y_s, paulis)

# Clean and factor using φ and ψ
Y_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    Y_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
Y_s_decomp_cleaned

{'I': 0,
 'X': sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2 + sin(s*theta)*cos(psi),
 'Y': -sin(psi)**2*sin(s*theta/2)**2*cos(2*phi) - sin(s*theta/2)**2*cos(psi)**2 + cos(s*theta/2)**2,
 'Z': 2*(sin(phi)*sin(s*theta/2)*cos(psi) - cos(phi)*cos(s*theta/2))*sin(psi)*sin(s*theta/2)}

In [14]:
Y_s_decomp_cleaned['X']


sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2 + sin(s*theta)*cos(psi)

In [15]:
Y_s_decomp_cleaned['Y']

-sin(psi)**2*sin(s*theta/2)**2*cos(2*phi) - sin(s*theta/2)**2*cos(psi)**2 + cos(s*theta/2)**2

In [16]:
Y_s_decomp_cleaned['Z']

2*(sin(phi)*sin(s*theta/2)*cos(psi) - cos(phi)*cos(s*theta/2))*sin(psi)*sin(s*theta/2)

## X(t) Component ##

In [17]:
X_s = U_dag * X * U
X_s = sp.simplify(X_s)
X_s = sp.expand_trig(X_s)
X_s = sp.trigsimp(X_s)
X_s

Matrix([
[      2*(sin(phi)*cos(s*theta/2) + sin(s*theta/2)*cos(phi)*cos(psi))*sin(psi)*sin(s*theta/2), (I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))**2 + exp(-2*I*phi)*sin(psi)**2*sin(s*theta/2)**2],
[(I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))**2 + exp(2*I*phi)*sin(psi)**2*sin(s*theta/2)**2,       -2*(sin(phi)*cos(s*theta/2) + sin(s*theta/2)*cos(phi)*cos(psi))*sin(psi)*sin(s*theta/2)]])

In [18]:
X_s_decomp = pauli_decomposition(X_s, paulis)
X_s_decomp = sp.simplify(X_s_decomp)
X_s_decomp = sp.trigsimp(X_s_decomp)
X_s_decomp

{I: 0, X: exp(2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2 - sin(s*theta/2)**2*cos(psi)**2 + cos(s*theta/2)**2 + exp(-2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2, Y: -I*exp(2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2 - sin(s*theta)*cos(psi) + I*exp(-2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2, Z: 2*(sin(phi)*cos(s*theta/2) + sin(s*theta/2)*cos(phi)*cos(psi))*sin(psi)*sin(s*theta/2)}

In [19]:
X_s_decomp = pauli_decomposition(X_s, paulis)

# Clean and factor using φ and ψ
X_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    X_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
X_s_decomp_cleaned

{'I': 0,
 'X': sin(psi)**2*sin(s*theta/2)**2*cos(2*phi) - sin(s*theta/2)**2*cos(psi)**2 + cos(s*theta/2)**2,
 'Y': sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2 - sin(s*theta)*cos(psi),
 'Z': 2*(sin(phi)*cos(s*theta/2) + sin(s*theta/2)*cos(phi)*cos(psi))*sin(psi)*sin(s*theta/2)}

In [20]:
X_s_decomp_cleaned['X']

sin(psi)**2*sin(s*theta/2)**2*cos(2*phi) - sin(s*theta/2)**2*cos(psi)**2 + cos(s*theta/2)**2

In [21]:
X_s_decomp_cleaned['Y']

sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2 - sin(s*theta)*cos(psi)

In [22]:
X_s_decomp_cleaned['Z']

2*(sin(phi)*cos(s*theta/2) + sin(s*theta/2)*cos(phi)*cos(psi))*sin(psi)*sin(s*theta/2)

## Checking I components

In [23]:
I_s = U_dag * I2 * U
I_s = sp.simplify(I_s)
I_s = sp.expand_trig(I_s)
I_s = sp.trigsimp(I_s)
I_s

Matrix([
[1, 0],
[0, 1]])

In [24]:
I_s_decomp = pauli_decomposition(I_s, paulis)

# Clean and factor using φ and ψ
I_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    I_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
I_s_decomp_cleaned

{'I': 1, 'X': 0, 'Y': 0, 'Z': 0}

# Relaxation Contributions

## $ \sigma_- $ Channel
$$ \sigma^- = (X-iY)/2 $$

In [25]:
sigma_minus = (X - sp.I * Y) / 2
sigma_minus

Matrix([
[0, 0],
[1, 0]])

In [26]:

sigma_minus_s  = U_dag * sigma_minus * U
sigma_minus_s  = sp.simplify(sigma_minus_s )
sigma_minus_s  = sp.expand_trig(sigma_minus_s )
sigma_minus_s  = sp.trigsimp(sigma_minus_s )
sigma_minus_s 

Matrix([
[I*(I*sin(phi) - cos(phi))*(I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))*sin(psi)*sin(s*theta/2),                                                     exp(-2*I*phi)*sin(psi)**2*sin(s*theta/2)**2],
[                                               (I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))**2, -I*(I*sin(phi) - cos(phi))*(I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))*sin(psi)*sin(s*theta/2)]])

In [27]:
sigma_minus_s_decomp = pauli_decomposition(sigma_minus_s , paulis)
sigma_minus_s_decomp = sp.simplify(sigma_minus_s_decomp)
sigma_minus_s_decomp = sp.trigsimp(sigma_minus_s_decomp)
sigma_minus_s_decomp

{I: 0, X: (I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))**2/2 + exp(-2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2, Y: -I*(I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))**2/2 + I*exp(-2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2, Z: I*(I*sin(phi) - cos(phi))*(I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))*sin(psi)*sin(s*theta/2)}

In [28]:
sigma_minus_s_decomp = pauli_decomposition(sigma_minus_s , paulis)

# Clean and factor using φ and ψ
sigma_minus_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    sigma_minus_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
sigma_minus_s_decomp_cleaned

{'I': 0,
 'X': (sin(s*theta/2)**2*cos(2*phi)/2 + sin(s*theta/2)**2/2)*sin(psi)**2 + I*(-sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2/2 - sin(s*theta)*cos(psi)/2) - sin(s*theta/2)**2 + 1/2,
 'Y': I*((sin(s*theta/2)**2*cos(2*phi)/2 - sin(s*theta/2)**2/2)*sin(psi)**2 + sin(s*theta/2)**2 - 1/2) + sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2/2 - sin(s*theta)*cos(psi)/2,
 'Z': ((sin(phi)*cos(s*theta/2) + sin(s*theta/2)*cos(phi)*cos(psi))*sin(s*theta/2) + I*(-sin(phi)*sin(s*theta/2)*cos(psi) + cos(phi)*cos(s*theta/2))*sin(s*theta/2))*sin(psi)}

In [29]:
sigma_minus_s_decomp_cleaned['X']

(sin(s*theta/2)**2*cos(2*phi)/2 + sin(s*theta/2)**2/2)*sin(psi)**2 + I*(-sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2/2 - sin(s*theta)*cos(psi)/2) - sin(s*theta/2)**2 + 1/2

In [30]:
sigma_minus_s_decomp_cleaned['Y']

I*((sin(s*theta/2)**2*cos(2*phi)/2 - sin(s*theta/2)**2/2)*sin(psi)**2 + sin(s*theta/2)**2 - 1/2) + sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2/2 - sin(s*theta)*cos(psi)/2

In [31]:
sigma_minus_s_decomp_cleaned['Z']

((sin(phi)*cos(s*theta/2) + sin(s*theta/2)*cos(phi)*cos(psi))*sin(s*theta/2) + I*(-sin(phi)*sin(s*theta/2)*cos(psi) + cos(phi)*cos(s*theta/2))*sin(s*theta/2))*sin(psi)

## $ \sigma_+ $ Channel
$$ \sigma^+ = (X+iY)/2 $$

In [32]:
sigma_plus = (X + sp.I * Y) / 2
sigma_plus

Matrix([
[0, 1],
[0, 0]])

In [33]:

sigma_plus_s  = U_dag * sigma_plus * U
sigma_plus_s  = sp.simplify(sigma_plus_s )
sigma_plus_s  = sp.expand_trig(sigma_plus_s )
sigma_plus_s  = sp.trigsimp(sigma_plus_s )
sigma_plus_s 

Matrix([
[-I*(I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))*exp(I*phi)*sin(psi)*sin(s*theta/2),                                   (I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))**2],
[                                        exp(2*I*phi)*sin(psi)**2*sin(s*theta/2)**2, I*(I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))*exp(I*phi)*sin(psi)*sin(s*theta/2)]])

In [34]:
sigma_plus_s_decomp = pauli_decomposition(sigma_plus_s , paulis)
sigma_plus_s_decomp = sp.simplify(sigma_plus_s_decomp)
sigma_plus_s_decomp = sp.trigsimp(sigma_plus_s_decomp)
sigma_plus_s_decomp

{I: 0, X: (I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))**2/2 + exp(2*I*phi)*sin(psi)**2*sin(s*theta/2)**2/2, Y: I*((I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))**2 - exp(2*I*phi)*sin(psi)**2*sin(s*theta/2)**2)/2, Z: (sin(s*theta/2)*cos(psi) - I*cos(s*theta/2))*exp(I*phi)*sin(psi)*sin(s*theta/2)}

In [35]:
sigma_plus_s_decomp = pauli_decomposition(sigma_plus_s , paulis)

# Clean and factor using φ and ψ
sigma_plus_s_decomp_cleaned = fully_simplify_pauli_decomposition(
    sigma_plus_s_decomp,
    collect_vars=[phi, psi, sp.sin(phi), sp.cos(phi), sp.sin(psi), sp.cos(psi)]
)
sigma_plus_s_decomp_cleaned

{'I': 0,
 'X': (sin(s*theta/2)**2*cos(2*phi)/2 + sin(s*theta/2)**2/2)*sin(psi)**2 + I*(sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2/2 + sin(s*theta)*cos(psi)/2) - sin(s*theta/2)**2 + 1/2,
 'Y': I*(-sin(psi)**2*sin(s*theta/2)**2*cos(2*phi)/2 - sin(s*theta/2)**2*cos(psi)**2/2 + cos(s*theta/2)**2/2) + sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2/2 - sin(s*theta)*cos(psi)/2,
 'Z': ((sin(phi)*cos(s*theta/2) + sin(s*theta/2)*cos(phi)*cos(psi))*sin(s*theta/2) + I*(sin(phi)*sin(s*theta/2)*cos(psi) - cos(phi)*cos(s*theta/2))*sin(s*theta/2))*sin(psi)}

In [36]:
sigma_plus_s_decomp_cleaned['X']

(sin(s*theta/2)**2*cos(2*phi)/2 + sin(s*theta/2)**2/2)*sin(psi)**2 + I*(sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2/2 + sin(s*theta)*cos(psi)/2) - sin(s*theta/2)**2 + 1/2

In [37]:
sigma_plus_s_decomp_cleaned['Y']

I*(-sin(psi)**2*sin(s*theta/2)**2*cos(2*phi)/2 - sin(s*theta/2)**2*cos(psi)**2/2 + cos(s*theta/2)**2/2) + sin(2*phi)*sin(psi)**2*sin(s*theta/2)**2/2 - sin(s*theta)*cos(psi)/2

In [38]:
sigma_plus_s_decomp_cleaned['Z']

((sin(phi)*cos(s*theta/2) + sin(s*theta/2)*cos(phi)*cos(psi))*sin(s*theta/2) + I*(sin(phi)*sin(s*theta/2)*cos(psi) - cos(phi)*cos(s*theta/2))*sin(s*theta/2))*sin(psi)



---

### Compute the variance over time

We define the variance of this process as:

$$
\mathrm{Var} = \int_0^1 f(s)^2 \, ds
$$

This integral gives us a symbolic expression that depends on θ, φ, and ψ. It represents the **Itô variance** associated with the stochastic evolution.

---

### Sample from the variance

Once we evaluate the variance for specific values of θ, φ, and ψ, we obtain a real scalar σ². We can sample from the corresponding zero-mean Gaussian distribution:

$$
x \sim \mathcal{N}(0, \mathrm{Var})
$$

This sampled value represents one instance of the quantum noise coefficient aligned with this Pauli component.

---

### Construct the Hermitian noise matrix

Once we sample all 3 Pauli components (X, Y, Z), we form the Hermitian matrix:

$$
I_d = e_d \begin{bmatrix}
Z_c & X_c - i Y_c \\
X_c + i Y_c & -Z_c
\end{bmatrix}
$$

where `e_d` is the depolarizing amplitude, and $ X_c, Y_c, Z_c $ are the sampled components.

This matrix can be added to a unitary evolution or inserted into a stochastic noise model.



### Deterministic Relaxation Contribution

In addition to the stochastic contributions from amplitude damping, there is a deterministic component arising from the average (non-stochastic) part of the Lindblad evolution.

This is captured by the term:
$$
\Lambda := -\frac{\epsilon_1^2}{2} \int_0^1 \left( L_{\downarrow,s}^\dagger L_{\downarrow,s} \right) ds,
$$
which leads to a Hermitian drift term added to the noisy gate.

The corresponding integrals are:
$$
\begin{aligned}
\text{det}_1 &= \frac{\theta - \sin(\theta)}{2\theta}, \\
\text{det}_2 &= \frac{1 - \cos(\theta)}{\theta}, \\
\text{det}_3 &= \frac{\theta + \sin(\theta)}{2\theta},
\end{aligned}
$$

and they form the matrix:
$$
\texttt{deterministic} = -\frac{\epsilon_1^2}{2}
\begin{pmatrix}
\text{det}_1 & \frac{i}{2} e^{-i\phi} \text{det}_2 \\
-\frac{i}{2} e^{i\phi} \text{det}_2 & \text{det}_3
\end{pmatrix}
$$


In [49]:


# Define symbols
s, theta, phi = sp.symbols('s theta phi', real=True)
epsilon_1 = sp.symbols('epsilon_1', real=True)

#theta = sp.pi  # Arbitrary phase, can be set to pi/2
# Y - gate
phi_val = sp.pi/2  # Z gate is a rotation of pi/2 around the Z axis
psi_val = 0

sigma_minus_s_sub = sigma_minus_s.subs({phi:phi_val, psi:psi_val})

# Hermitian conjugate
sigma_minus_s_dag_sub = sigma_minus_s_sub.H

# Compute L†L and simplify
product = sp.simplify(sigma_minus_s_dag_sub * sigma_minus_s_sub)

# Integrate each element of the product matrix from s = 0 to 1
integrated = sp.Matrix(2, 2, lambda i, j: sp.simplify(sp.integrate(product[i, j], (s, 0, 1))))

# Extract raw det1, det3 directly (diagonal)
det1 = sp.simplify(integrated[1, 1])  # lower-right
det3 = sp.simplify(integrated[0, 0])  # upper-left

# Compute the off-diagonal term
off_diag = sp.simplify(integrated[0, 1])

# Define the known complex prefactor explicitly
#prefactor = -sp.I / 2 * sp.exp(-sp.I * phi)

# Isolate the real-valued part of det2 by factoring out the prefactor
# Multiply both sides symbolically instead of dividing
det2 = sp.simplify(sp.trigsimp(off_diag))
det2 = sp.simplify(sp.trigsimp(det2.rewrite(sp.cos)))


# Construct Λ matrix in canonical Hermitian form
Λ = (-epsilon_1**2)/2 * sp.Matrix([
    [det1,  sp.I/2 * sp.exp(-sp.I * phi) * det2],
    [-sp.I/2 * sp.exp(sp.I * phi) * det2, det3]
])

# Display results
display(sp.Eq(sp.Symbol('det1'), det1))
display(sp.Eq(sp.Symbol('det2'), det2))
display(sp.Eq(sp.Symbol('det3'), det3))
display(sp.Symbol('Λ ='), Λ)

Λ.subs({phi:phi_val})


Eq(det1, 0)

Eq(det2, 0)

Eq(det3, 1)

Λ =

Matrix([
[0,               0],
[0, -epsilon_1**2/2]])

Matrix([
[0,               0],
[0, -epsilon_1**2/2]])

### Variances and covariances for relaxation Itô processes depending on Z(t)
**Dephasing Noise — $ \sigma_z $ Channel**
Pure dephasing (phase damping) is modeled via a Lindblad operator:
$$
L_\phi(s) = \sqrt{\frac{1}{T_2}} \, \sigma_z(s),
$$
where $ \sigma_z(s) $ is the Pauli-Z operator in the interaction picture.

As with other stochastic terms, we expand this in second-order Itô integrals. The statistical quantities used to build the covariance matrix are:
$$
\begin{aligned}
\mathbb{V}[\mathcal{I}_1] &= \frac{2\theta + \sin(2\theta)}{4\theta}, \\
\mathbb{V}[\mathcal{I}_2] &= \frac{2\theta - \sin(2\theta)}{4\theta}, \\
\text{Cov}[\mathcal{I}_1, \mathcal{I}_2] &= \frac{\sin^2(\theta)}{2\theta}.
\end{aligned}
$$

This results in a 2×2 Hermitian contribution matrix $ \texttt{Ip} $, scaled by the dephasing amplitude:
$$
\epsilon_\phi = \sqrt{\frac{1}{2} \left( \frac{t_g}{T_2} - \frac{t_g}{2T_1} \right)}.
$$

In [ ]:
Z_s_decomp 



{'I': 0,
 'X': I*((I*sin(phi) - cos(phi))*(I*sin(s*theta/2)*cos(psi) + cos(s*theta/2)) - (I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))*exp(I*phi))*sin(psi)*sin(s*theta/2),
 'Y': (-(I*sin(phi) - cos(phi))*(I*sin(s*theta/2)*cos(psi) + cos(s*theta/2)) + (-I*sin(s*theta/2)*cos(psi) + cos(s*theta/2))*exp(I*phi))*sin(psi)*sin(s*theta/2),
 'Z': -2*sin(psi)**2*sin(s*theta/2)**2 + 1}

In [ ]:
# Compute variances/covariance
def var_and_cov(decomp):
    fz = decomp['Z']
    fy = decomp['Y']
    var_z = sp.integrate(fz**2, (s, 0, 1))
    var_y = sp.integrate(fy**2, (s, 0, 1))
    cov_zy = sp.integrate(fz * fy, (s, 0, 1))
    return fz, fy, sp.simplify(var_z), sp.simplify(var_y), sp.simplify(cov_zy)

fZz, fZy, var_Z, var_Y, cov_ZY = var_and_cov(Z_s_decomp)

fZz = sp.simplify(fZz)
fZy = sp.simplify(fZy)

var_Z = sp.trigsimp(var_Z, trig=True)
#var_Z = var_Z.rewrite(sp.sin(2*theta))  # Force rewrite if needed
var_Z = sp.simplify(var_Z)

Var_Z2_sin2theta = sp.trigsimp(var_Y, trig=True)
#Var_Z2_sin2theta = Var_Z2_sin2theta.rewrite(sp.sin(2*theta))  # Force rewrite if needed
Var_Z2_sin2theta = sp.simplify(Var_Z2_sin2theta)

# Display results
display(sp.Eq(sp.Symbol("Var_Z1"), var_Z))
display(sp.Eq(sp.Symbol("Var_Z2"), Var_Z2_sin2theta))
display(sp.Eq(sp.Symbol("Cov_Z12"), cov_ZY))

display(sp.Eq(sp.Symbol("f_Zz"), fZz))
display(sp.Eq(sp.Symbol("f_Zy"), fZy))

Eq(Var_Z1, Piecewise(((3*theta*sin(psi)**4/2 - 2*theta*sin(psi)**2 + theta - 2*sin(psi)**4*sin(theta) + sin(psi)**4*sin(2*theta)/4 + 2*sin(psi)**2*sin(theta))/theta, (theta > 0) | (theta < 0)), (1, True)))

Eq(Var_Z2, Piecewise(((-4*theta*(-3*exp(4*I*phi) + 6*exp(2*I*phi) - 3*exp(8*I*psi) + 4*exp(6*I*psi) - 2*exp(4*I*psi) + 4*exp(2*I*psi) - 2*exp(4*I*(phi + psi)) + 8*exp(2*I*(phi + psi)) - 3*exp(4*I*(phi + 2*psi)) - 28*exp(2*I*(phi + 2*psi)) + 8*exp(2*I*(phi + 3*psi)) + 6*exp(2*I*(phi + 4*psi)) + 4*exp(2*I*(2*phi + psi)) + 4*exp(2*I*(2*phi + 3*psi)) - 3)*exp(I*(16*phi + 32*psi + 3*theta)) + I*(24*(-exp(5*I*(2*phi + 3*psi)) + exp(3*I*(2*phi + 5*psi)) - exp(I*(6*phi + 13*psi)) + exp(I*(6*phi + 17*psi)) - exp(I*(6*phi + 19*psi)) + exp(I*(10*phi + 13*psi)) - exp(I*(10*phi + 17*psi)) + exp(I*(10*phi + 19*psi)))*exp(I*(8*phi + 16*psi + 3*theta)) + (8*(-exp(10*I*(phi + 2*psi)) - 4*exp(8*I*(phi + 2*psi)) - exp(6*I*(phi + 2*psi)) + 2*exp(5*I*(2*phi + 3*psi)) + 2*exp(4*I*(2*phi + 3*psi)) + 2*exp(4*I*(2*phi + 5*psi)) - 2*exp(3*I*(2*phi + 5*psi)) + 2*exp(2*I*(3*phi + 8*psi)) - exp(2*I*(3*phi + 10*psi)) - exp(2*I*(5*phi + 6*psi)) + 2*exp(2*I*(5*phi + 8*psi)) + 2*exp(I*(6*phi + 13*psi)) - 2*exp(I*(6*ph

Eq(Cov_Z12, Piecewise(((4*theta*(3*exp(2*I*phi) + 3*exp(8*I*psi) + 2*exp(6*I*psi) - 2*exp(2*I*psi) + 2*exp(2*I*(phi + psi)) - 2*exp(2*I*(phi + 3*psi)) - 3*exp(2*I*(phi + 4*psi)) - 3)*exp(I*(8*phi + 32*psi + 3*theta)) + I*(4*(7*exp(5*I*(phi + 3*psi)) + 7*exp(3*I*(phi + 5*psi)) + 3*exp(I*(3*phi + 13*psi)) - 7*exp(I*(3*phi + 17*psi)) - 3*exp(I*(3*phi + 19*psi)) + 3*exp(I*(5*phi + 13*psi)) - 7*exp(I*(5*phi + 17*psi)) - 3*exp(I*(5*phi + 19*psi)))*exp(I*(4*phi + 16*psi + 3*theta)) + (8*(-exp(5*I*(phi + 3*psi)) + exp(5*I*(phi + 4*psi)) + exp(3*I*(phi + 4*psi)) - exp(3*I*(phi + 5*psi)) - exp(I*(3*phi + 13*psi)) + exp(I*(3*phi + 17*psi)) + exp(I*(3*phi + 19*psi)) - exp(I*(3*phi + 20*psi)) - exp(I*(5*phi + 12*psi)) - exp(I*(5*phi + 13*psi)) + exp(I*(5*phi + 17*psi)) + exp(I*(5*phi + 19*psi)))*exp(I*theta) + ((-6*exp(5*I*(phi + 3*psi)) + exp(5*I*(phi + 4*psi)) + exp(3*I*(phi + 4*psi)) - 6*exp(3*I*(phi + 5*psi)) + 2*exp(3*I*(phi + 6*psi)) + 2*exp(I*(3*phi + 13*psi)) - 2*exp(I*(3*phi + 14*psi)) + 6

Eq(f_Zz, -2*sin(psi)**2*sin(s*theta/2)**2 + 1)

Eq(f_Zy, -((I*sin(phi) - cos(phi))*(I*sin(s*theta/2)*cos(psi) + cos(s*theta/2)) + (I*sin(s*theta/2)*cos(psi) - cos(s*theta/2))*exp(I*phi))*sin(psi)*sin(s*theta/2))